In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda
import time

print("⚠️  CUDA C++ is the PRIMARY learning material!")

---

## Part 1: Mixed Precision Fundamentals

### Why Mixed Precision?

```
Full FP32 Training:
┌─────────────────────────────────────────┐
│  Weights: FP32    Gradients: FP32       │
│  Activations: FP32   Loss: FP32         │
│  Memory: 100%    Speed: 1x              │
└─────────────────────────────────────────┘

Mixed Precision Training:
┌─────────────────────────────────────────┐
│  Master Weights: FP32 (kept for update) │
│  Forward/Backward: FP16 (fast compute)  │
│  Loss Scaling: prevent underflow        │
│  Memory: ~50%    Speed: 2-8x            │
└─────────────────────────────────────────┘
```

### The Problem: Gradient Underflow

```
FP16 representable range: ~6e-8 to 65504

Small gradients (common in deep networks):
  gradient = 1e-7  →  FP16 = 0 (underflow!)

Solution: Loss Scaling
  1. Scale loss by large factor (e.g., 1024)
  2. Gradients also scale by 1024
  3. Unscale before weight update
  
  gradient = 1e-7 * 1024 = 1.024e-4  ✓ Representable!
```

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile mixed_precision.cu
// mixed_precision.cu - Mixed precision training primitives
#include <stdio.h>
#include <cuda_runtime.h>
#include <cuda_fp16.h>
#include <curand_kernel.h>

// Convert FP32 weights to FP16 for forward pass
__global__ void fp32_to_fp16(float* fp32, half* fp16, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        fp16[idx] = __float2half(fp32[idx]);
    }
}

// Convert FP16 gradients to FP32 and unscale
__global__ void fp16_to_fp32_unscale(
    half* fp16_grad, float* fp32_grad, float scale, int n
) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        fp32_grad[idx] = __half2float(fp16_grad[idx]) / scale;
    }
}

// Check for inf/nan in gradients (for dynamic loss scaling)
__global__ void check_overflow(half* grads, int* has_overflow, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float val = __half2float(grads[idx]);
        if (isinf(val) || isnan(val)) {
            atomicExch(has_overflow, 1);
        }
    }
}

// FP16 forward pass (example: linear layer + ReLU)
__global__ void linear_relu_fp16(
    half* input, half* weight, half* bias, half* output,
    int batch, int in_features, int out_features
) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;  // batch
    int col = blockIdx.x * blockDim.x + threadIdx.x;  // out_features
    
    if (row < batch && col < out_features) {
        float sum = __half2float(bias[col]);
        
        for (int k = 0; k < in_features; k++) {
            sum += __half2float(input[row * in_features + k]) *
                   __half2float(weight[k * out_features + col]);
        }
        
        // ReLU
        sum = fmaxf(sum, 0.0f);
        output[row * out_features + col] = __float2half(sum);
    }
}

// Scale loss for FP16 backward pass
__global__ void scale_loss(half* loss, half* scaled_loss, float scale, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float val = __half2float(loss[idx]) * scale;
        scaled_loss[idx] = __float2half(val);
    }
}

// SGD update with unscaled gradients
__global__ void sgd_update(
    float* weights, float* gradients, float lr, int n
) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        weights[idx] -= lr * gradients[idx];
    }
}

int main() {
    printf("=== Mixed Precision Training Primitives ===\n\n");
    
    // Demonstrate loss scaling effect
    float small_gradient = 1e-7f;
    half fp16_direct = __float2half(small_gradient);
    
    float scale = 1024.0f;
    float scaled_gradient = small_gradient * scale;
    half fp16_scaled = __float2half(scaled_gradient);
    float unscaled = __half2float(fp16_scaled) / scale;
    
    printf("Gradient underflow example:\n");
    printf("  Original gradient: %e\n", small_gradient);
    printf("  Direct FP16:       %e (underflow!)\n", __half2float(fp16_direct));
    printf("  Scaled (x1024):    %e\n", scaled_gradient);
    printf("  After unscale:     %e (preserved!)\n\n", unscaled);
    
    // Memory savings
    int params = 100000000;  // 100M parameters
    printf("Memory savings for %dM parameters:\n", params/1000000);
    printf("  FP32 only:      %.1f GB\n", params * 4.0 / 1e9);
    printf("  Mixed (FP16):   %.1f GB\n", params * 2.0 / 1e9);
    printf("  Savings:        %.1f%%\n", 50.0);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o mixed_precision mixed_precision.cu
!./mixed_precision

---

## Part 2: Dynamic Loss Scaling

### Automatic Scale Adjustment

```
Dynamic Loss Scaling Algorithm:

1. Start with large scale (e.g., 65536)
2. For each training step:
   a. Scale loss
   b. Backward pass in FP16
   c. Check gradients for inf/nan
   d. If overflow:
      - Skip weight update
      - Reduce scale by factor (e.g., /2)
   e. If no overflow:
      - Unscale gradients
      - Update weights
      - Optionally increase scale

Benefits:
  - Automatically finds optimal scale
  - Handles varying gradient magnitudes
  - Robust across different models
```

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile dynamic_scaling.cu
// dynamic_scaling.cu - Dynamic loss scaling
#include <stdio.h>
#include <cuda_runtime.h>
#include <cuda_fp16.h>

class DynamicLossScaler {
public:
    float scale;
    float scale_factor;
    int scale_window;
    int steps_since_increase;
    
    DynamicLossScaler(
        float init_scale = 65536.0f,
        float factor = 2.0f,
        int window = 2000
    ) : scale(init_scale), scale_factor(factor), 
        scale_window(window), steps_since_increase(0) {}
    
    void update(bool overflow) {
        if (overflow) {
            // Reduce scale
            scale /= scale_factor;
            steps_since_increase = 0;
            printf("  Overflow detected! Scale reduced to %.1f\n", scale);
        } else {
            steps_since_increase++;
            if (steps_since_increase >= scale_window) {
                // Try increasing scale
                scale *= scale_factor;
                steps_since_increase = 0;
                printf("  Increasing scale to %.1f\n", scale);
            }
        }
    }
};

// Check for overflow in gradients
__global__ void check_grads(half* grads, int* overflow_flag, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float val = __half2float(grads[idx]);
        if (isinf(val) || isnan(val)) {
            atomicExch(overflow_flag, 1);
        }
    }
}

// Simulate training step with scaled gradients
__global__ void simulate_backward(
    half* grad_out, float scale, int n, float base_grad
) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float grad = base_grad * scale;  // Scaled gradient
        grad_out[idx] = __float2half(grad);
    }
}

int main() {
    printf("=== Dynamic Loss Scaling Demo ===\n\n");
    
    const int n = 1000;
    DynamicLossScaler scaler;
    
    half* d_grads;
    int* d_overflow;
    int h_overflow;
    
    cudaMalloc(&d_grads, n * sizeof(half));
    cudaMalloc(&d_overflow, sizeof(int));
    
    // Simulate training steps with different gradient magnitudes
    float gradient_sizes[] = {1e-4f, 1e-5f, 1e-6f, 1e-7f, 1.0f, 100.0f};
    int num_steps = sizeof(gradient_sizes) / sizeof(float);
    
    for (int step = 0; step < num_steps; step++) {
        float base_grad = gradient_sizes[step];
        printf("Step %d: Base gradient = %e, Scale = %.1f\n", 
               step, base_grad, scaler.scale);
        
        // Simulate backward with current scale
        cudaMemset(d_overflow, 0, sizeof(int));
        simulate_backward<<<(n+255)/256, 256>>>(d_grads, scaler.scale, n, base_grad);
        check_grads<<<(n+255)/256, 256>>>(d_grads, d_overflow, n);
        
        cudaMemcpy(&h_overflow, d_overflow, sizeof(int), cudaMemcpyDeviceToHost);
        
        scaler.update(h_overflow == 1);
        printf("\n");
    }
    
    cudaFree(d_grads);
    cudaFree(d_overflow);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o dynamic_scaling dynamic_scaling.cu
!./dynamic_scaling

---

## Part 3: Operations That Need FP32

### Numerically Sensitive Operations

```
Keep in FP32:
├── Softmax (large exponentials)
├── LayerNorm / BatchNorm (variance computation)
├── Loss functions (cross-entropy, MSE)
├── Weight updates (accumulation)
└── Reduction operations (sum, mean)

Can use FP16:
├── Linear layers (matmul) ← Tensor Cores!
├── Convolutions ← Tensor Cores!
├── Element-wise operations (ReLU, GELU)
└── Pooling
```

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile safe_ops.cu
// safe_ops.cu - FP32-safe operations
#include <stdio.h>
#include <cuda_runtime.h>
#include <cuda_fp16.h>
#include <float.h>

// Softmax: Must use FP32 internally to avoid overflow
__global__ void softmax_fp16_safe(
    half* input, half* output, int batch, int dim
) {
    int b = blockIdx.x;
    if (b >= batch) return;
    
    // Find max (for numerical stability)
    float max_val = -FLT_MAX;
    for (int i = threadIdx.x; i < dim; i += blockDim.x) {
        float val = __half2float(input[b * dim + i]);
        max_val = fmaxf(max_val, val);
    }
    
    // Warp reduction for max
    for (int offset = 16; offset > 0; offset >>= 1) {
        max_val = fmaxf(max_val, __shfl_down_sync(0xffffffff, max_val, offset));
    }
    max_val = __shfl_sync(0xffffffff, max_val, 0);
    
    // Compute exp and sum (in FP32!)
    float sum = 0.0f;
    for (int i = threadIdx.x; i < dim; i += blockDim.x) {
        float val = __half2float(input[b * dim + i]);
        sum += expf(val - max_val);  // FP32 exp
    }
    
    // Warp reduction for sum
    for (int offset = 16; offset > 0; offset >>= 1) {
        sum += __shfl_down_sync(0xffffffff, sum, offset);
    }
    sum = __shfl_sync(0xffffffff, sum, 0);
    
    // Output (can be FP16)
    for (int i = threadIdx.x; i < dim; i += blockDim.x) {
        float val = __half2float(input[b * dim + i]);
        float softmax = expf(val - max_val) / sum;
        output[b * dim + i] = __float2half(softmax);
    }
}

// LayerNorm: Variance needs FP32 precision
__global__ void layernorm_fp16_safe(
    half* input, half* output, half* gamma, half* beta,
    int batch, int dim, float eps
) {
    int b = blockIdx.x;
    if (b >= batch) return;
    
    // Compute mean (FP32)
    float sum = 0.0f;
    for (int i = threadIdx.x; i < dim; i += blockDim.x) {
        sum += __half2float(input[b * dim + i]);
    }
    for (int offset = 16; offset > 0; offset >>= 1) {
        sum += __shfl_down_sync(0xffffffff, sum, offset);
    }
    float mean = __shfl_sync(0xffffffff, sum, 0) / dim;
    
    // Compute variance (FP32)
    float var_sum = 0.0f;
    for (int i = threadIdx.x; i < dim; i += blockDim.x) {
        float val = __half2float(input[b * dim + i]) - mean;
        var_sum += val * val;
    }
    for (int offset = 16; offset > 0; offset >>= 1) {
        var_sum += __shfl_down_sync(0xffffffff, var_sum, offset);
    }
    float var = __shfl_sync(0xffffffff, var_sum, 0) / dim;
    float rstd = rsqrtf(var + eps);  // FP32 rsqrt
    
    // Normalize and output (can be FP16)
    for (int i = threadIdx.x; i < dim; i += blockDim.x) {
        float val = __half2float(input[b * dim + i]);
        float norm = (val - mean) * rstd;
        float g = __half2float(gamma[i]);
        float b_val = __half2float(beta[i]);
        output[b * dim + i] = __float2half(norm * g + b_val);
    }
}

int main() {
    printf("=== FP32-Safe Operations for Mixed Precision ===\n\n");
    
    printf("Operations requiring FP32 internally:\n");
    printf("  ✓ Softmax (exp can overflow)\n");
    printf("  ✓ LayerNorm (variance precision)\n");
    printf("  ✓ BatchNorm (statistics)\n");
    printf("  ✓ Cross-entropy loss\n");
    printf("  ✓ Gradient accumulation\n");
    printf("\n");
    printf("Safe for FP16 throughout:\n");
    printf("  ✓ Matrix multiply (Tensor Cores)\n");
    printf("  ✓ Convolution (Tensor Cores)\n");
    printf("  ✓ ReLU, GELU activations\n");
    printf("  ✓ Pooling\n");
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o safe_ops safe_ops.cu
!./safe_ops

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
# Python demonstration: precision issues in softmax
import numpy as np

def softmax_precision_demo():
    """Show why softmax needs FP32."""
    print("Softmax Precision Demo")
    print("=" * 50)
    
    # Large values that cause overflow in FP16
    x_fp32 = np.array([1000.0, 1001.0, 1002.0], dtype=np.float32)
    x_fp16 = x_fp32.astype(np.float16)
    
    print(f"\nInput values: {x_fp32}")
    
    # FP16 exp overflow
    print(f"\nFP16 exp results:")
    print(f"  exp(1000) in FP16: {np.exp(np.float16(1000.0))}")
    
    # Safe softmax with max subtraction
    x_shifted = x_fp32 - x_fp32.max()
    print(f"\nAfter subtracting max: {x_shifted}")
    print(f"  exp values: {np.exp(x_shifted)}")
    print(f"  softmax: {np.exp(x_shifted) / np.exp(x_shifted).sum()}")

softmax_precision_demo()

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

In [ ]:
%%writefile mixed_precision_exercises.cu
// CUDA C++ Mixed Precision Exercises
// Week 13, Day 3: Mixed Precision Training

#include <cuda_runtime.h>
#include <cuda_fp16.h>
#include <mma.h>
#include <stdio.h>

using namespace nvcuda;

// Exercise 1: FP32 to FP16 conversion kernel
// TODO: Implement efficient conversion with proper rounding

// Exercise 2: Loss scaling implementation
// TODO: Scale gradients to prevent underflow

// Exercise 3: Mixed precision GEMM
// TODO: FP16 inputs, FP32 accumulation, FP16 output

int main() {
    printf("Mixed Precision Exercises - Implement your solutions above\n");
    printf("Compile with: nvcc -arch=sm_70 mixed_precision_exercises.cu -o mixed_precision_exercises\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_70 mixed_precision_exercises.cu -o mixed_precision_exercises && ./mixed_precision_exercises

### 🔶 Python/CuPy Exercises (Alternative)

In [ ]:
# Exercise plan
def mixed_precision_gemm_plan():
    print("Mixed Precision GEMM Implementation")
    print("=" * 50)
    print()
    print("Forward Pass:")
    print("  1. Convert FP32 weights to FP16")
    print("  2. WMMA matmul with FP16 inputs")
    print("  3. Accumulate in FP32")
    print("  4. Output in FP16 for next layer")
    print()
    print("Backward Pass:")
    print("  1. Scale loss")
    print("  2. Compute gradients in FP16")
    print("  3. Check for overflow")
    print("  4. Unscale gradients")
    print("  5. Update FP32 master weights")

mixed_precision_gemm_plan()

---

## Summary

### Mixed Precision Recipe

```
1. Master weights in FP32
2. Forward/backward in FP16 (Tensor Cores)
3. Loss scaling to prevent underflow
4. FP32 for sensitive operations
5. Dynamic scaling for robustness
```

### Performance Guidelines

| Technique | Memory Savings | Speedup |
|-----------|---------------|--------|
| Pure FP32 | 0% | 1x |
| FP16 compute only | ~20% | 2-4x |
| Full mixed precision | ~50% | 2-8x |

### Tomorrow: cuBLAS with Tensor Cores
Use library-level Tensor Core acceleration.